In [4]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/CSE519/project/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/CSE519/project


In [25]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.0 MB/s eta 0:00:00


In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import cross_val_score, KFold, train_test_split

from sklearn.linear_model import LinearRegression

from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error

import xgboost as xgb
from catboost import CatBoostRegressor
import lightgbm as lgb

In [9]:
df = pd.read_csv('train.csv')
df

,Pop_Total,Net_Migration,Mortality_Rate,Fertility_Rate,Pop_Density,Female_Pop,Electricity,Mobile_Users,Land_Area,GDP,...,Political_Stability,Count_Athlete,Count_Medal,Count_Male,Count_Female,Urban_Pop_Ratio,Pop_15_64_Ratio,Pop_0_14_Ratio,Foreign_Invest_GDP_Ratio,Is_Summer
0,-0.205162,-0.018251,5.228675,2.237272,-0.186208,-0.921303,-2.498267,-0.746072,-0.078526,-0.234474,...,-2.623937,16,0,16,0,-0.242192,-0.274324,0.000005,0.031342,1
1,-0.199973,0.006590,5.228675,2.249444,-0.185439,-0.792672,-2.498267,-0.746072,-0.078526,-0.234244,...,-2.623937,8,0,8,0,-0.242126,-0.275203,0.000004,0.031359,1
2,-0.193656,-0.014213,4.835618,2.282088,-0.184503,-0.690210,-2.498267,-0.746072,-0.078526,-0.233740,...,-2.623937,5,0,5,0,-0.242072,-0.276205,0.000004,0.031433,1
3,-0.186301,-0.055216,4.447632,2.331883,-0.183413,-0.609194,-2.498267,-0.746072,-0.078526,-0.233545,...,-2.623937,8,0,8,0,-0.242028,-0.277135,0.000004,0.031496,1
4,-0.177807,-6.727063,3.643766,2.406577,-0.182155,-0.468287,-2.498267,-0.746072,-0.078526,-0.231749,...,-2.623937,11,0,11,0,-0.241790,-0.278074,0.000004,0.031202,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2558,-0.180113,-0.902232,0.374205,0.276436,-0.173746,1.033527,-1.429173,-0.641642,-0.217509,-0.227855,...,-1.428562,14,3,9,5,-0.239772,-0.277407,0.000003,0.031187,1
2559,-0.177356,-0.808909,0.460424,0.296354,-0.173057,1.126325,-1.363458,-0.438778,-0.217509,-0.229075,...,-1.436314,16,4,8,8,-0.239963,-0.277732,0.000003,0.031295,1
2560,-0.172295,-0.403200,0.222054,0.474511,-0.171793,1.206448,-1.168787,1.277035,-0.217509,-0.217927,...,-0.995933,9,0,4,5,-0.240224,-0.278215,0.000003,0.031482,1
2561,-0.168115,-0.365485,0.125692,0.400371,-0.170749,1.183686,-1.532420,1.068886,-0.217509,-0.215837,...,-0.929835,2,0,2,0,-0.240393,-0.278465,0.000003,0.031535,0


In [10]:
df.columns

Index(['Pop_Total', 'Net_Migration', 'Mortality_Rate', 'Fertility_Rate',
       'Pop_Density', 'Female_Pop', 'Electricity', 'Mobile_Users', 'Land_Area',
       'GDP', 'GDP_PCAP', 'Inflation', 'GINI', 'Military_Expense',
       'Unemployment', 'Labor_Force', 'Labor_Female', 'Primary_Completion',
       'Literacy_Rate', 'Political_Stability', 'Count_Athlete', 'Count_Medal',
       'Count_Male', 'Count_Female', 'Urban_Pop_Ratio', 'Pop_15_64_Ratio',
       'Pop_0_14_Ratio', 'Foreign_Invest_GDP_Ratio', 'Is_Summer'],
      dtype='object')

In [12]:
features = ['Pop_Total', 'Net_Migration', 'Mortality_Rate', 'Fertility_Rate', 'Pop_Density', 'Female_Pop', 'Electricity', 'Mobile_Users', 'Land_Area', 'GDP', 'GDP_PCAP', 'Inflation', 'GINI', 'Military_Expense', 'Unemployment', 'Labor_Force', 'Labor_Female', 'Primary_Completion', 'Literacy_Rate', 'Political_Stability', 'Urban_Pop_Ratio', 'Pop_15_64_Ratio', 'Pop_0_14_Ratio', 'Foreign_Invest_GDP_Ratio', 'Is_Summer']

targets = ['Count_Athlete', 'Count_Medal', 'Count_Male', 'Count_Female']

X = df[features]
y = df[targets[0]]

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [30]:
model = xgb.XGBRegressor()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

mse = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mse}")

feature_importances = model.feature_importances_
print("Feature Importances:")
for feature, importance in zip(X.columns, feature_importances):
    print(f"{feature}: {importance}")

Mean Squared Error: 1673.0524826201724
Mean Absolute Error: 20.631070806958927
Feature Importances:
Pop_Total: 0.012000885792076588
Net_Migration: 0.005094857420772314
Mortality_Rate: 0.004901843145489693
Fertility_Rate: 0.01486271247267723
Pop_Density: 0.005336599424481392
Female_Pop: 0.014372753910720348
Electricity: 0.061220623552799225
Mobile_Users: 0.003982461988925934
Land_Area: 0.08851205557584763
GDP: 0.3629010319709778
GDP_PCAP: 0.01352665200829506
Inflation: 0.002652920549735427
GINI: 0.030847277492284775
Military_Expense: 0.006374741904437542
Unemployment: 0.0036439350806176662
Labor_Force: 0.06877249479293823
Labor_Female: 0.04414904862642288
Primary_Completion: 0.007295611780136824
Literacy_Rate: 0.01078184787184
Political_Stability: 0.06713156402111053
Urban_Pop_Ratio: 0.009572957642376423
Pop_15_64_Ratio: 0.013634636998176575
Pop_0_14_Ratio: 0.05184231325984001
Foreign_Invest_GDP_Ratio: 0.009576704353094101
Is_Summer: 0.08701146394014359


In [31]:
model = CatBoostRegressor()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

mse = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mse}")

feature_importances = model.feature_importances_
print("Feature Importances:")
for feature, importance in zip(X.columns, feature_importances):
    print(f"{feature}: {importance}")

Learning rate set to 0.04586
0:	learn: 108.6569615	total: 10.2ms	remaining: 10.2s
1:	learn: 105.6321672	total: 16.7ms	remaining: 8.34s
2:	learn: 102.3616098	total: 23.6ms	remaining: 7.83s
3:	learn: 99.4699438	total: 32.2ms	remaining: 8.01s
4:	learn: 96.8758341	total: 38.8ms	remaining: 7.72s
5:	learn: 94.4119931	total: 45.1ms	remaining: 7.47s
6:	learn: 92.1103506	total: 52ms	remaining: 7.38s
7:	learn: 89.6970843	total: 58.6ms	remaining: 7.27s
8:	learn: 87.2674326	total: 65.2ms	remaining: 7.18s
9:	learn: 85.0926515	total: 72.1ms	remaining: 7.13s
10:	learn: 83.0602464	total: 79ms	remaining: 7.1s
11:	learn: 80.9392870	total: 88.5ms	remaining: 7.29s
12:	learn: 79.0914382	total: 95.8ms	remaining: 7.28s
13:	learn: 77.3269386	total: 109ms	remaining: 7.7s
14:	learn: 75.5816295	total: 116ms	remaining: 7.64s
15:	learn: 74.0468574	total: 124ms	remaining: 7.61s
16:	learn: 72.5750276	total: 131ms	remaining: 7.6s
17:	learn: 71.1652902	total: 138ms	remaining: 7.55s
18:	learn: 69.5778398	total: 146ms	r

In [32]:
model = lgb.LGBMRegressor()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

mse = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mse}")

feature_importances = model.feature_importances_
print("Feature Importances:")
for feature, importance in zip(X.columns, feature_importances):
    print(f"{feature}: {importance}")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000621 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5930
[LightGBM] [Info] Number of data points in the train set: 2050, number of used features: 25
[LightGBM] [Info] Start training from score 66.517561
Mean Squared Error: 1506.991935348197
Mean Absolute Error: 19.54572062032105
Feature Importances:
Pop_Total: 90
Net_Migration: 207
Mortality_Rate: 112
Fertility_Rate: 210
Pop_Density: 125
Female_Pop: 116
Electricity: 62
Mobile_Users: 100
Land_Area: 176
GDP: 239
GDP_PCAP: 73
Inflation: 99
GINI: 131
Military_Expense: 137
Unemployment: 75
Labor_Force: 73
Labor_Female: 107
Primary_Completion: 86
Literacy_Rate: 42
Political_Stability: 116
Urban_Pop_Ratio: 79
Pop_15_64_Ratio: 70
Pop_0_14_Ratio: 186
Foreign_Invest_GDP_Ratio: 142
Is_Summer: 147
